Import de la bibliothèque Ollama

In [1]:
import ollama

Téléchargement du modèle Llama 3.1

In [2]:
ollama.pull('llama3.1:8b')

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

Génération de texte avec la fonction generate() 

In [ ]:
result=ollama.generate(model = 'llama3.1:8b',
    prompt='Give me a joke on Generative AI',
    )

print(result['response']) #afficher un seul message 

Génération de texte en mode conversation (chat())

In [ ]:
response = ollama.chat( model='llama3.1:8b',messages = [
    {
        'role': 'user',
        'content': 'Give me a joke on Generative AI'
    }
]
)
print(response['message']['content']) #supporte plusieurs messages dans une conversation

Création du modèle Jarvis

In [ ]:
modelfiles='''
FROM llama3.1:8b
SYSTEM You are Jarvis form Iron and the user is Tony Stark.
'''

ollama.create(model='jarvis2', modelfile=modelfile)

In [ ]:
ollama.delete('jarvis2')

Rest API : 
Interaction avec un modèle via Client

In [ ]:
from ollama import Client #classe client permet de communiquer avec le serveur ollama 
client = Client(host= 'http://localhost:11434')
response = client.chat(model='llama3.1:8b', messages=[{"role": "user", "content": "Explain gravity to a 6 year old kid?"}])

response 

In [ ]:
from ollama import Client 
client = Client(host= 'http://localhost:11434')
response = client.chat(model='llama3.1:8b', messages=[
    {"role": "system", "content": "You are Jarvis from Iron Man and the user is Tony Stark. Respond in only a single line."},
    {"role": "user", "content": "Hi"},
    ])

response['message']['content']

In [ ]:
from openai import OpenAI

llm = OpenAI(base_url='http://localhost:11434', api_key='blank') #api key est vide car on utilise le serveur local d'ollama

response = llm.chat.completions.create(
    model='llama3.1:8b',
    messages=[
        {"role": "system", "content": "You are Jarvis from Iron man and the user is Tony Stark. Respond in only a single line."},
        {"role": "user", "content": "Hi"},
        {"role": "assistant", "content": "Good morning, Mr. Stark. Shall I proceed with your scheduler for today?"},
        {"role": "user", "content": "Yes, run the diagnostics and tell me the results in a single line."},
    ]
)
print(response.choices[0].message.content)

In [ ]:
pip install langchain
pip install langchain-core
pip install langchain-Ollama
pip install langchain-community

LangChain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that gives a one line definition of the word entered by user."),
    ("user", " {word}?"),
])
messages = chat_template.format_messages(word="Sesquipedalian")
messages

In [ ]:
from langchain_ollama import ChatOllama
llm=ChatOllama(model="llama3.1:latest",
               temperature= 0)

In [ ]:
ai_message = llm.invoke(messages)
ai_message

In [ ]:
from langchain_core.output_parsers import StrOutputParser
chain= chat_template | llm | StrOutputParser()

In [ ]:
chain.invoke({"word":"Onomatopoeia"})

RAG Application using Ollama and Langchain

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama

In [ ]:
raw_documents = TextLoader("./LangchainRetrieval.txt").load()

raw_documents 

Découper le texte en morceaux

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,   # nombre de caractères par chunk
    chunk_overlap=20  # chevauchement entre chunks
)
documents = text_splitter.split_documents(raw_documents)

len(documents)
print(documents[0])
print(documents[1])

Créer les embeddings avec Ollama

In [ ]:
oembed = OllamaEmbeddings(base_url='http://localhost:11434', model="nomic-embed-text")

In [ ]:
db = Chroma.from_documents(documents, embedding=oembed)

In [ ]:
query = "What is text embedding and how does langchain help in doing it"
docs = db.similarity_search(query)

len(docs)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassThrough

In [ ]:
template = """Answer the question based on the following context: {context} \n Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [ ]:
model = ChatOllama(model="llama3.1:latest", temperature=0)

In [ ]:
retriever = db.as_retriever()

In [ ]:
def format_doc(docs): #transforme les documents pertinents en un seul texte.
    return "\n".join([doc.page_content for doc in docs])

In [ ]:
chain = (
    {"context": retriever | format_doc, "question": RunnablePassThrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [ ]:
chain.invoke("What is text embedding and how does langchain help in doing it")